In [ ]:
# SOURCE This notebook was coded with help from ChatGPT

In [5]:
import pandas as pd
import json
from tqdm.auto import tqdm
from pathlib import Path

from gpt4all import GPT4All

In [9]:
data_dir: Path = Path("../../../data")
responses_dir: Path = Path("responses")

In [10]:
data_dir

WindowsPath('../../../data')

In [11]:
df = pd.read_json(data_dir / Path("dataset_english.json"))
model = GPT4All("nous-hermes-llama2-13b.Q4_0.gguf")

In [12]:
pii = {}

with open(data_dir / Path("pii_class_counts.json"), "r") as file:
    pii = json.load(file)

english_piis = list(pii[0].keys())[:-1]

In [19]:
instruction = (
    "You are searching for personal info in a given text."
    + "\n Respond by stating all personal information in json format. Personal information can be the following: \n"
    + str(english_piis)
    + "\n Example: Hello Margareth, how are you? Here is my IBAN DE05 8903 8742 12342 32."
    + "\n Response: {'FIRSTNAME': 'Margareth', 'IBAN':'DE05 8903 8742 12342 32'}"
)

In [20]:
print(instruction)

You are searching for personal info in a given text.
 Respond by stating all personal information in json format. Personal information can be the following: 
['PREFIX', 'FIRSTNAME', 'LASTNAME', 'DATE', 'TIME', 'PHONEIMEI', 'USERNAME', 'GENDER', 'CITY', 'STATE', 'URL', 'JOBAREA', 'EMAIL', 'JOBTYPE', 'COMPANYNAME', 'JOBTITLE', 'STREET', 'SECONDARYADDRESS', 'COUNTY', 'AGE', 'USERAGENT', 'ACCOUNTNAME', 'ACCOUNTNUMBER', 'CURRENCYSYMBOL', 'AMOUNT', 'CREDITCARDISSUER', 'CREDITCARDNUMBER', 'CREDITCARDCVV', 'PHONENUMBER', 'SEX', 'IP', 'ETHEREUMADDRESS', 'BITCOINADDRESS', 'MIDDLENAME', 'IBAN', 'VEHICLEVRM', 'DOB', 'PIN', 'CURRENCY', 'PASSWORD', 'CURRENCYNAME', 'LITECOINADDRESS', 'CURRENCYCODE', 'BUILDINGNUMBER', 'ORDINALDIRECTION', 'MASKEDNUMBER', 'ZIPCODE', 'BIC', 'IPV4', 'IPV6', 'MAC', 'NEARBYGPSCOORDINATE', 'VEHICLEVIN', 'EYECOLOR', 'HEIGHT', 'SSN']
 Example: Hello Margareth, how are you? Here is my IBAN DE05 8903 8742 12342 32.
 Response: {'FIRSTNAME': 'Margareth', 'IBAN':'DE05 8903 8742 123

In [21]:
def execute_batch(df: pd.DataFrame, model):
    response = []
    with model.chat_session(instruction, "### Instruction:\n{0}\n### Response:\n"):
        for item in tqdm(df.iloc(), total=len(df)):
            response.append(
                model.generate(
                    f"This is the text with personal info: '{item.unmasked_text}'",
                    temp=0,
                    n_batch=128,
                )
            )
    return response

In [22]:
start = 3004
batch_size = 7


for i in tqdm(
    range(batch_size + start, len(df), batch_size), total=len(df) // batch_size
):
    batch = {}

    low = i - batch_size
    up = i

    slice = df.iloc[low:up]

    batch["response"] = execute_batch(slice, model)
    batch["low"] = low
    batch["up"] = up

    with open(responses_dir / Path(f"response-{low}.json"), mode="w") as file:
        file.write(json.dumps(batch))


else:
    low = len(df) - len(df) % 1000
    up = len(df)

    batch = {}

    slice = df.iloc[low:up]

    batch["response"] = execute_batch(slice, model)
    batch["low"] = low
    batch["up"] = up

    with open(responses_dir / Path(f"response-{low}.json"), mode="w") as file:
        file.write(json.dumps(batch))

  0%|          | 0/6214 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]